In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy.io import loadmat
from preprocess_methods import CausalIndividualLevel
from pathlib import Path
from causal_discovery.algos.notears import NoTears
import networkx as nx
import hashlib
import os

In [2]:
cells = []

In [3]:
GS3 = sio.loadmat('../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/Aged_cells/HPPC_MS/HPPC_MultiSine_GS3.mat', struct_as_record=False, simplify_cells=True)
Y1= sio.loadmat('../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/Aged_cells/HPPC_MS/HPPC_MultiSine_Y1.mat', struct_as_record=False, simplify_cells=True)
cells = [['GS3)', GS3], ['Y1' , Y1]]

OGS3 = sio.loadmat('../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/Aged_cells/Pseudo_OCV/OCVDis_GS3.mat', struct_as_record=False, simplify_cells=True)
OY1 = sio.loadmat('../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/Aged_cells/Pseudo_OCV/OCVDis_Y1.mat', struct_as_record=False, simplify_cells=True)
#cells = [['OGS3', OGS3], ['OY1' , OY1]]

In [4]:
files = [f'P{i}' for i in range(2, 21)]        # gives ['P2', 'P3', ..., 'P20']

base_path = '../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/NMC_cells/HPPC_MS/HPPC_MultiSine_'
for p in files:
    new_data = sio.loadmat(base_path + f'{p}.mat', struct_as_record=False, simplify_cells=True)
    cells.append([str(p), new_data])

In [5]:
files = [f'F{i}' for i in range(1, 19)]        # gives ['F1', 'F2', ..., 'F18']

base_path = '../../Data/imbalanced performance/Full factorial design of experiments dataset for parallel-connected lithium-ion cells imbalanced performance investigation/Parallel-connected module experimental campaign/1_Single_cell_characterisation/NCA_cells/HPPC_MS/HPPC_MultiSine_'
for f in files:
    new_data = sio.loadmat(base_path + f'{f}.mat', struct_as_record=False, simplify_cells=True)
    cells.append([str(f), new_data])

In [6]:
cell_data = []
for cell in cells:
    cell_info = {
        'CurrentData': cell[1]['CurrentData'], 
        'VoltageData': cell[1]['VoltageData'],
        'TempData': cell[1]['TempData'],
        'Step_Index': cell[1]['StepIndex'],
        'CycleIndex' : cell[1]['CycleIndex'],
        'TimeData' : cell[1]['TimeData']
    }
    cell_data.append(cell_info)

cell_dataset = pd.DataFrame(cell_data)
print(cell_dataset.head())

                                         CurrentData  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                         VoltageData  \
0  [4.197761058807373, 4.197635173797607, 4.19782...   
1  [4.198263168334961, 4.1982808113098145, 4.1982...   
2  [4.1984028816223145, 4.198349475860596, 4.1983...   
3  [4.19843864440918, 4.198358058929443, 4.198379...   
4  [4.198364734649658, 4.1983642578125, 4.1983065...   

                                            TempData  \
0  [24.041702270507812, 23.95568084716797, 23.926...   
1  [23.43080711364746, 23.43080711364746, 23.4161...   
2  [22.298236846923828, 22.298236846923828, 22.29...   
3  [23.2643928553498, 23.26439666748047, 23.26439...   
4  [22.531064987182617, 22.54578399658203, 22.

In [7]:
def flatten_array(arr):
    # Works for arrays like [[x], [y], [z]]
    return np.array([x[0] if isinstance(x, (list, np.ndarray)) else x for x in arr])

def make_cell_dataframe(row, cols):
    clean = {}

    # Flatten all nested arrays
    for name in cols:
        clean[name] = flatten_array(row[name])

    # Build long-format dataframe
    return pd.DataFrame(clean)

In [8]:
def splitDataForEachByStep(data, cols, groups):
    cell_list = []
    for idx, row in data.iterrows():
        #cell_info = {name : row[name] for name in cols}
        #new_pd = pd.DataFrame([cell_info])
        cell_df = make_cell_dataframe(row, cols)
        obs_datasets = []
        interventional_datasets = []
        for i in range(len(groups)):
            subset = cell_df[cell_df["Step_Index"].isin(groups[i])].reset_index(drop=True)

            if i == 0:
                obs_datasets.append(subset)

            interventional_datasets.append(subset)
        cell_list.append([obs_datasets, interventional_datasets])
    return cell_list
        


In [9]:
groups = [[11, 13, 16, 18, 21], [9, 14], [10, 16], [12], [17], [19], [20], [22]]
cols = ['Step_Index', 'TempData', 'VoltageData', 'CurrentData']
sub_df_1 = splitDataForEachByStep(cell_dataset, cols, groups)
print(sub_df_1[1][1][0])


       Step_Index   TempData  VoltageData  CurrentData
0              11  23.226440     3.658872   -18.192619
1              11  23.226440     3.658149   -17.971931
2              11  23.226440     3.656804   -17.875931
3              11  23.226440     3.652112   -17.928795
4              11  23.226440     3.644859   -18.089554
...           ...        ...          ...          ...
17610          18  23.357729     3.168857     0.000000
17611          18  23.357729     3.168854     0.000000
17612          18  23.320255     3.168962     0.000000
17613          18  23.320255     3.169020     0.000000
17614          18  23.320255     3.168992     0.000000

[17615 rows x 4 columns]


In [10]:
multicausaldataset = CausalIndividualLevel.multienvcausaldiscoverydataset(cell_dataset, cols, groups)
print(multicausaldataset)

          TempData  VoltageData  CurrentData  Group
0        24.041702     4.197761          0.0    1.0
1        23.955681     4.197635          0.0    1.0
2        23.926302     4.197824          0.0    1.0
3        23.979719     4.197635          0.0    1.0
4        24.001940     4.197635          0.0    1.0
...            ...          ...          ...    ...
2788247  22.857420     3.319781          0.0    7.0
2788248  22.857420     3.319725          0.0    7.0
2788249  22.857420     3.319683          0.0    7.0
2788250  22.857420     3.319831          0.0    7.0
2788251  22.857420     3.319834          0.0    7.0

[2788252 rows x 4 columns]


In [ ]:
CausalIndividualLevel.multienvcausaldiscovery(multicausaldataset, '../../Src/structures/1cells_char/images/multienv')

In [11]:
CausalIndividualLevel.multienvcausaldiscoveryChat(multicausaldataset, '../../Src/structures/1cells_char/images/multienv/chat_dagma')

  0%|          | 0/6000 [00:00<?, ?it/s]

Learned adjacency matrix:
 [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]
Adjacency matrix:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [ ]:
CausalIndividualLevel.multienvcausaldiscoverypergroup(multicausaldataset, '../../Src/structures/1cells_char/images/multienv/per_group')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaled_cells = []
for cell in cells:
    CurrentData = np.asarray(cell[1]["CurrentData"]).flatten()
    VoltageData = np.asarray(cell[1]["VoltageData"]).flatten()
    StepIndex    = np.asarray(cell[1]["StepIndex"]).flatten()
    TempData    = np.asarray(cell[1]["TempData"]).flatten()
    df = pd.DataFrame({
        "Voltage_Data": VoltageData,
        "Current_Data": CurrentData,
        "Step_Index": StepIndex,
        "Temp_Data": TempData,
    })
    scaler = StandardScaler()
    scaled = scaler.fit_transform(df.values)
    scaled_df = pd.DataFrame(scaled, columns=df.columns)
    scaled_cells.append(scaled_df)

print(scaled_cells)
#cell_dataset = pd.DataFrame(cell_data)
#print(cell_dataset.head())

In [ ]:
pd_concat = pd.concat(scaled_cells, ignore_index=True)
CausalIndividualLevel.run_bigdata_notears(pd_concat, '../../Src/structures/1cells_char/images/multienv/chat_with_step')

In [ ]:
def observational_struct_learning(cell_list, obs_or_int, save_point, sort_by="Step_Index", type="cell"):
    os.makedirs(save_point, exist_ok=True)

    dag_counter = {}  # counts identical DAGs

    for idx, cell in enumerate(cell_list):

        # Create folder for this cell
        cell_dir = os.path.join(save_point, f"{type}_{idx}")
        os.makedirs(cell_dir, exist_ok=True)

        for group_idx, group in enumerate(cell[obs_or_int]):

            df_cell = group.copy()

            # ---- skip empty ----
            if df_cell.empty:
                continue

            # ---- drop Step_Index ----
            if sort_by in df_cell.columns:
                df_cell = df_cell.drop(columns=[sort_by])

            # ---- drop zero-variance columns ----
            df_cell = df_cell.loc[:, df_cell.nunique() > 1]
            if df_cell.shape[1] < 2:
                continue

            # Prepare data
            X = df_cell.to_numpy().astype(float)
            cols = df_cell.columns.tolist()

            # ---- run NoTears ----
            no_tears_alg = NoTears(rho=1, alpha=0.1, l1_reg=0, lr=1e-2)
            no_tears_alg.learn(X)

            W_est = no_tears_alg.get_result()
            dag = (np.abs(W_est) > 0.3).astype(int)

            # ---- count DAG ----
            dag_hash = hashlib.md5(dag.tobytes()).hexdigest()
            dag_counter[dag_hash] = dag_counter.get(dag_hash, 0) + 1

            # ---- create folder for this group ----
            group_dir = os.path.join(cell_dir, f"group_{group_idx}")
            os.makedirs(group_dir, exist_ok=True)

            # ---- save adjacency matrix ----
            mat_path = os.path.join(group_dir, "dag.npy")
            np.save(mat_path, dag)

            # ---- draw DAG ----
            G = nx.DiGraph()
            for i, src in enumerate(cols):
                for j, tgt in enumerate(cols):
                    if dag[i, j] == 1:
                        G.add_edge(src, tgt)

            plt.figure(figsize=(7, 6))
            pos = nx.spring_layout(G, seed=42)
            nx.draw(G, pos, with_labels=True, node_size=2000, font_size=10, arrows=True)
            plt.title(f"Cell {idx} – Group {group_idx} – Learned DAG")

            img_path = os.path.join(group_dir, "dag.png")
            plt.savefig(img_path, dpi=200, bbox_inches="tight")
            plt.close()

            print(f"Saved DAG → cell {idx} / group {group_idx}")

    # ---- save summary at the end only ----
    summary_path = os.path.join(save_point, "dag_summary.txt")
    with open(summary_path, "w") as f:
        for h, count in dag_counter.items():
            f.write(f"{h}: {count}\n")

    print("\nDAG summary saved to:", summary_path)
    print("Unique DAGs found:", len(dag_counter))



  
                

In [ ]:
save_point = '../../Src/structures/1cells_char/images/obs'
observational_struct_learning(sub_df_1, 0, save_point)

In [ ]:
save_point = '../../Src/structures/1cells_char/images/int'
observational_struct_learning(sub_df_1, 1, save_point)

In [ ]:
def load_and_visualize_unique_dags(save_point):
    """
    Reads dag_summary.txt, reconstructs unique DAGs from stored dag.npy files,
    and shows them visually with their counts.
    """

    # ---- Load summary ----
    summary_path = os.path.join(save_point, "dag_summary.txt")
    if not os.path.exists(summary_path):
        print("ERROR: No dag_summary.txt found in:", save_point)
        return

    summary = {}
    with open(summary_path, "r") as f:
        for line in f:
            h, count = line.strip().split(": ")
            summary[h] = int(count)

    unique_dags = {}

    # ---- Traverse folder structure ----
    for cell in os.listdir(save_point):
        cell_path = os.path.join(save_point, cell)
        if not os.path.isdir(cell_path):
            continue

        for group in os.listdir(cell_path):
            group_path = os.path.join(cell_path, group)
            dag_path = os.path.join(group_path, "dag.npy")

            if not os.path.exists(dag_path):
                continue

            dag = np.load(dag_path)
            dag_hash = hashlib.md5(dag.tobytes()).hexdigest()

            # Only keep DAGs mentioned in the summary
            if dag_hash in summary:
                unique_dags[dag_hash] = dag

    # ---- Visualization ----
    print("Found", len(unique_dags), "unique DAGs.")

    n = len(unique_dags)
    plt.figure(figsize=(6 * n, 6))

    for i, (dag_hash, dag) in enumerate(unique_dags.items(), 1):
        G = nx.DiGraph()
        num_nodes = dag.shape[0]
        nodes = [f"X{i}" for i in range(num_nodes)]
        G.add_nodes_from(nodes)

        for s in range(num_nodes):
            for t in range(num_nodes):
                if dag[s, t] == 1:
                    G.add_edge(nodes[s], nodes[t])

        plt.subplot(1, n, i)
        pos = nx.spring_layout(G, seed=0)
        nx.draw(
            G,
            pos,
            with_labels=True,
            node_size=2000,
            font_size=10,
            arrows=True
        )
        plt.title(f"DAG {i}\nHash: {dag_hash[:6]}\nCount: {summary[dag_hash]}")

    plt.tight_layout()
    plt.show()

    return unique_dags, summary


In [ ]:
save_point_obs = '../../Src/structures/1cells_char/images/obs'
obs_unique_dags, obs_summary = load_and_visualize_unique_dags(save_point_obs)
#x0 = temprature, x1 = voltage, x2 = current (amper)

In [ ]:
save_point_int = '../../Src/structures/1cells_char/images/int'
int_unique_dags, int_summary = load_and_visualize_unique_dags(save_point_int)
#x0 = temprature, x1 = voltage, x2 = current (amper)

In [ ]:
# example: real variable names from preprocessing step
node_names = ["TempData", "Voltage", "Current"] # <-- IMPORTANT

edge_counts = CausalIndividualLevel.count_edge_frequencies(int_unique_dags, int_summary, node_names)

for edge, count in sorted(edge_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{edge[0]} → {edge[1]}: {count} occurrences")
